# Import module

In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

# Make dataframe

In [22]:
bike_path='data/'
bike_list=os.listdir(bike_path)
df=[]
for file in bike_list:
    file_path=os.path.join(bike_path,file)
    file_name='bike_'+file_path.split('.')[0][-4:]
    file_name=pd.read_csv(file_path,low_memory=False)
    df.append(file_name)

In [ ]:
weather_path='weather/'
weather_list=os.listdir(weather_path)
weather_df=[]
for file in weather_list:
    file_path=os.path.join(weather_path,file)
    file_name='weather_'+file_path.split('.')[0][-4:]
    file_name=pd.read_xlsx(file_path)
    weather_df.append(file_name)

In [31]:
df[4].head()

,대여일시,대여대여소,대여대여소이름,대여거치대,반납일시,반납대여소,반납대여소이름,반납거치대,이용시간,이용거리
0,2018-01-01 00:00:10,'227',' 양평2나들목 보행통로 입구',9,2018-01-01 00:04:03,'227',' 양평2나들목 보행통로 입구',9,3,50.0
1,2018-01-01 00:00:42,'2219',"' 고속터미널역 8-1번, 8-2번 출구 사이'",15,2018-01-01 00:25:41,'3511',' 응봉역 1번출구',9,24,7670.0
2,2018-01-01 00:00:58,'567',' 성수역 2번출구 앞',13,2018-01-01 00:09:33,'3506',' 영동대교 북단',2,8,1390.0
3,2018-01-01 00:01:02,'568',' 청계8가사거리 부근',5,2018-01-01 00:10:13,'521',' 왕십리역 11번 출구 앞',9,8,1820.0
4,2018-01-01 00:01:03,'361',' 동묘앞역 1번출구 뒤',4,2018-01-01 00:06:18,'383',' 신당역 12번 출구 뒤',8,4,850.0
